In [ ]:
# run the following command in terminal to login
# huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ^C
object address  : 0x1047c9fc0
object refcount : 3
object type     : 

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("abhinand/MedEmbed-large-v0.1")

sentences = [
    "Patient shows symptoms of myocardial infarction.",
    "Blood pressure is within normal range."
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

model.safetensors:   2%|2         | 31.5M/1.34G [00:00<?, ?B/s]

In [ ]:
import json
import re


from sklearn.metrics.pairwise import cosine_similarity

with open("../data/guideline_db.json") as f:
    db = json.load(f)

vector_store = (
    list()
)

# Step 1 - build vector store
for chunk in db:
    text = chunk["text"]
    embedding = embedder.encode(text)
    vector_store.append({"text": text, "embedding": embedding, "metadata": chunk["metadata"]})

In [3]:
vector_store[0]

{'text': "Canadian Network for Mood and Anxiety Treatments (CANMAT) 2023Update on Clinical Guidelines for Management of Major DepressiveDisorder in Adults: Réseau canadien pour les traitements del'humeur et de l'anxiété (CANMAT) 2023 : Mise à jour des lignesdirectrices cliniques pour la prise en charge du troubledépressif majeur chez les adultes",
 'embedding': array([-1.62061173e-02,  5.23807704e-02,  1.72703676e-02,  9.71413106e-02,
        -6.41723722e-02,  6.88556358e-02, -2.10412126e-02,  5.95986880e-02,
         2.25023832e-03, -4.08442467e-02, -8.11044779e-03,  3.05131990e-02,
         3.71590406e-02, -4.17358428e-02,  1.27706677e-02,  4.44232300e-02,
         4.66283597e-02, -1.77929588e-02,  5.39403260e-02,  1.13438852e-01,
        -2.41671852e-03, -1.44715281e-02,  6.37220871e-03, -1.40065309e-02,
        -3.30602378e-02,  2.63231769e-02, -7.72838891e-02, -6.32242560e-02,
         6.36555403e-02,  8.51298347e-02,  4.37365435e-02,  9.14357901e-02,
         1.34799797e-02, -2.3

In [ ]:
def search(embedder, query, vector_store, k, min_similarity):
    query_embedding = embedder.encode(query.lower())
    # make magic happen here
    similarities = []
    results = []
    # calculate cosine similarity between each text and the query
    for i, chunk in enumerate(vector_store):
        similarity = cosine_similarity([query_embedding], [chunk["embedding"]])
        if similarity[0][0] >= min_similarity:
            similarities.append((i, similarity[0][0]))

    # sort the similarities based on similarity and select the top k
    similarities.sort(key=lambda x: x[1], reverse=True)
    for i, similarity in similarities[:k]:
            results.append(db[i])
    
    # if not results:
    #     return "No matching documents!"
    # print(results)
    if not results or not results[0]:
        return ["No matching documents!"]
    return results


# Test your search function!
query = "I have a patient who can’t take lithium but is currently acutely manic. Which medication should I try next?"
results = search(embedder,query, vector_store, 7, 0.1)

# write results to a file
with open("results.json", "w") as f:
    json.dump(results, f, indent=4)
# print results
# for i, result in enumerate(results):
#     print(f"Result {i+1}:")
#     print(f"Text: {result['text']}")
#     print(f"Metadata: {result['metadata']}")
#     print()